In [1]:
'''
This notebook should stay in root directory
'''
import os
import torch
import numpy as np
import pickle
from tokenizer import Tokenizer
from generation_manager import GeneratorManager


In [3]:
tokenizer = Tokenizer()
print('tokenizer loaded')

curr_model_path = 'ckpt/pretrain_progen_full.pth'#  model checkpoint
print('loading model from:', curr_model_path)
# Saving only state_dict:
#torch.save(model.state_dict(), model_state_dict_path)
#model_state_dict_path = load_model_path + 'model_only_state_dict_v0Last.pth'
#print('Model state dict saved')


tokenizer loaded
loading model from: ckpt/pretrain_progen_full.pth


In [4]:
# paths to the saved .p files

data_file = 'data_halogenase/dataset/hidro_test_filtered.p' # 

# Reload random_selection from the .p file
with open(data_file, "rb") as file:
    random_selection = pickle.load(file)

kws_file = 'data_specific_enzymes/databases_kws/pkl/3.8.1.pkl'

with open(kws_file, "rb") as file:
    kws_dictionary = pickle.load(file)

import random
# Print or use the reloaded datasets as needed
print("Random Selection:")

for key in random_selection:
    progen_kws = []
    if key in kws_dictionary:
        if kws_dictionary[key][-1] in tokenizer.taxa_to_ctrl_idx:
            id = tokenizer.taxa_to_ctrl_idx[kws_dictionary[key][-1]]
            progen_kws.append(id)
        for value in kws_dictionary[key][:-1]:
            if value in tokenizer.kw_to_ctrl_idx:
                id = tokenizer.kw_to_ctrl_idx[value]
                progen_kws.append(id)
    random_selection[key]['kw'] = [str(x) for x in progen_kws]

for key in random_selection:
    print(key)
    print(random_selection[key])
    print(len(random_selection))
    break


#print("---")
random_selection_reduced = []
random_selection_keys = random.sample(list(random_selection.keys()), len(random_selection))
for key in random_selection_keys:
    random_selection_reduced.append(random_selection[key])
print(len(random_selection_reduced))


Random Selection:
A0A7X5ZGM8
{'kw': ['8'], 'ex': 2, 'seq': 'MSYVDVGEGRPIVFLHGNPTWSYVWRNVIPYVKGNGRCLAPDLIGMGLSGLSPSGSYRFHDHVRYLDAWFGKMNLGEKIVLVSHDWGTALASYWASRYRSRVAGFAYMEGIIRPSRWSEFPDGMRKMFQALRSPVGRKMIMEENLFIEGIFPHTVIEKMDDETMNGYRMPYRDPERREAMLAWPRELPINGEPEEVADIVTSASEWIARSDIPKLLIAGDPGAALSGAALDYARSWSNQKEVKVKGSHVLQEDSPRDIGQALSEFIGNLP', 'rev': False, 'len': 270}
328
328


In [5]:
kws_dictionary

{'A5JTM5': [2, 903, 378, 72586],
 'D4Z2G1': [2, 216, 903, 378, 574, 1185, 452662],
 'O06652': [378, 59918],
 'P22643': [2, 216, 903, 378, 280],
 'P72156': [2, 963, 903, 378, 408, 479, 614, 47660],
 'Q1JU72': [2, 903, 378, 36773],
 'Q53464': [2, 378, 66693],
 'Q6NAM1': [2, 378, 1185, 258594],
 'Q9LCU3': [903, 378, 1667],
 'O85078': [903, 378, 1667],
 'P81594': [4, 1003, 903, 378, 408, 411, 472, 479, 677, 732, 272564],
 'Q51645': [2, 903, 378, 292],
 'Q52087': [378, 303],
 'Q60099': [2, 903, 378, 280],
 'A0A1L5BTC1': [216, 378, 574, 861109],
 'P0A3G2': [2, 216, 378, 614, 1829],
 'P0A3G3': [2, 216, 378, 1831],
 'P0A3G4': [216, 903, 378, 47881],
 'P24069': [378, 72586],
 'P24070': [378, 72586],
 'P59336': [2, 378, 269091],
 'P59337': [2, 378, 1185, 224911],
 'P60527': [378, 260551],
 'P9WMR9': [2, 7, 378, 1185, 83332],
 'P9WMS3': [378, 1185, 83332],
 'Q52086': [2, 378, 303],
 'Q52725': [903, 378, 36822],
 'Q59168': [378, 614, 85698],
 'Q59666': [378, 294],
 'Q59728': [378, 614, 303],
 'Q6Q

<p>To evaluate the generated sequence using teacher forcing:

Generate the Sequence: The transformer model with teacher forcing generates the complete sequence. 
At each time step, we provide the true input sequence (+ 1 versus previous step), as input to the model.

</p>

In [6]:
penalty = 0
topk = 1
generator = GeneratorManager(curr_model_path, penalty=penalty, topk=topk)

MODEL SIZE: 
1280
Found PyTorch checkpoint at  ckpt/pretrain_progen_full.pth
GPU aviable. Previous checkpoint loaded in GPU


In [7]:
predicted = []
true_tokens_index_in_probs_all = []
tokens_probs_all = []
true_value = []

In [8]:

for entry in random_selection_reduced:
    input_seq = entry['seq']
    keys = [int(item) for item in entry['kw']]
    
    # If testing general
    #keys = []

    res, tokens_prob, offset = generator.after_n_generation(input_seq, keys, 20)

    true_value.append(input_seq)
    true_tokens_index_in_prob = []
    true_tokens_index_in_probs_all.append(true_tokens_index_in_prob)
    tokens_probs_all.append(tokens_prob)
    predicted.append(input_seq[:offset] + res)
    

In [13]:
# testing teacher forcing new
true_value, predicted, tokens_probs_all = generator.techer_forcing_generation_new(data_file, testing_general = False)

In [13]:
print(len(predicted))

328


In [10]:
print(len(predicted[0]))
print(len(true_value[0]))
print(len(tokens_probs_all[0]))

292
292
272


In [11]:
print(predicted[0])
print(true_value[0])
print(len(tokens_probs_all[0]))

MFEAFESRRFAVDGAEIHARLARAARDLVGKPVALVFNGISHSVMMATPIDTEVFAREIALCRPLAHPDLPAIAAECPLDTVVEVPDAQIDERAINLHRGETWLWHSDAMYRTRRALRPLLAIERVPINPASGGCLMAEVTQALGGRTRPLYTAGLRLKHCALPDTQGQRSRNAYHLPLALHAIAFRPRDERLLQIVGNDLLHNAMMHPWRERQSHQLHGLLGELLGLALGEGAEPFDLPVPFLYGEDWLLSAHAYLQGAMRYRFSTREEALRLWRALAAAISGWVADLQHA
MFEAFESRRFAVDGAEIHARVGGQGAPLLLLHGHPQTHAMWHRVVPQLAQRFRIVAMDLRGYGDSSRPAPDDAHAMHGKRAMARDAVDVMAQLGHPRFAVCAHDRGARVAHRLALDHPQAVSRLLLLDIAPTLAMYEGTTDAFARAYWHWFTLIQPAPLPERMIGHDPVFYLRSVMGRRHAGLAAFDAAALAEYERCIALPGSVASICEDYRASATIDLDHDRADRAARRGIEAPLRVLWAEHGAVGRCFDVLALWRAVARDVSGRAVPCGHYIAEELPELLLAEMNDFLKD
272


In [14]:
# Create a directory named "ID_test_data" in the current working directory if it doesn't exist
data_dir = "Test_after20_k_1_general_model_EC3-8-1_KEYSCONVERTIONS_test_reduced"
query = "file"
os.makedirs(data_dir, exist_ok=True)

# TODO, salvare tokens_prob e true_tokens_index_in_prob
tokens_probs_data_file = os.path.join(data_dir, "tokens_probs_data_" + query + ".p")
with open(tokens_probs_data_file, "wb") as file:
    pickle.dump(tokens_probs_all, file)
    
true_tokens_index_data_file = os.path.join(data_dir, "true_tokens_index_data_" + query + ".p")
with open(true_tokens_index_data_file, "wb") as file:
    pickle.dump(true_tokens_index_in_probs_all, file)

# Save ID_test_data as a .p file
predicted_data_file = os.path.join(data_dir, "predicted_data_" + query + ".p")
with open(predicted_data_file, "wb") as file:
    pickle.dump(predicted, file)
    
# Save ID_test_data as a .p file
true_data_file = os.path.join(data_dir, "true_data_" + query + ".p")
with open(true_data_file, "wb") as file:
    pickle.dump(true_value, file)
torch.cuda.empty_cache()

<h3>Other things<h3>

In [ ]:
import ete3
ncbi = ete3.NCBITaxa()
# to update the taxonomic local database use:
# ncbi.update_taxonomy_database()
'''taxon_id = 505
lineage = ncbi.get_lineage(taxon_id)
print(lineage)
print(type(lineage))'''

tmp_dic = {}
for data in random_selection:
    sequence = data['sequence']['metadata']['sequence']
    length = data['sequence']['metadata']['length']
    identifier = data['sequence']['metadata']['accession'] 
    dics = {}
    dics['seq']=sequence
    dics['kw']=[0]
    dics['len']=int(length)
    tmp_dic[identifier]=dics

with open('data_lysozymes/random_selection_LYZOTEST_10Percent.p', "wb") as file:
    pickle.dump(tmp_dic, file)

In [ ]:
count = 0
inner_count = 0
for entry in random_selection:
    taxid = int(entry['metadata']['source_organism']['taxId']) # taxonomy id from NCBI
    inner_count = 0
    #print(taxid)
    try:
        tax_lineage1 = ncbi.get_lineage(taxid)
        # remove root taxonomy id
        tax_lineage1 = [element for element in tax_lineage1 if element not in [1, 2, 131567, 10239]]
        #tax_lineage2 = tokenizer.taxa_to_lineage[taxid] # make lineage in ncbi ids
        #print(tax_lineage1)
        res = []
        for ite in tax_lineage1:
            try:
                tmp = tokenizer.taxa_to_ctrl_idx[ite]
                res.append(tmp)
            except:
                inner_count += 1
        if res == [] and inner_count >= 1:
            count += 1
    except:
        count += 1
        print("FATAL taxa conversion")
        #tax_lineage = [tokenizer.taxa_to_ctrl_idx[ite] for ite in tax_lineage1] # now translated as ctrl code indices
print("Tax lineage problems:", count)
    # break

In [ ]:
#IF FILTERED DATASET: key retrieval

data_file = 'data_enzymes_classes/filtered_tests_new/EC7.p'
with open(data_file, "rb") as file:
    random_selection = pickle.load(file)

new_random_selection = []
new_rand_dict = {}


data_file_test_withkey_file = 'data_enzymes_classes/all_families_data/test_ec_7.p'
with open(data_file_test_withkey_file, "rb") as file:
    data_file_test_withkey = pickle.load(file)
    

#print(len(data_file_test_withkey))
#print(data_file_test_withkey['A6T228'])


test = []
import random
print(len(random_selection))
for key in data_file_test_withkey:
    if data_file_test_withkey[key]['seq'] in random_selection:
        test.append(data_file_test_withkey[key]['seq'])
        new_random_selection.append(data_file_test_withkey[key])
        new_rand_dict[key] = data_file_test_withkey[key]
for i in random_selection:
    if i not in test:
        print(i)

#saving test file as pickle
with open('data_enzymes_classes/filtered_tests/EC7_formatted.p', "wb") as file:
    random_selection = pickle.dump(new_rand_dict, file)

#print("---")
random_selection_reduced = new_random_selection
print(len(random_selection_reduced))
    
#print("Filtered Data:")
#for entry in filtered_data:
#    print(entry["sequence"])

In [ ]:

for entry in random_selection_reduced:
    input_seq = entry['seq']
    #print(input_seq)
    '''
    taxid = int(entry['metadata']['source_organism']['taxId']) # taxonomy id from NCBI
    try:
        tax_lineage = ncbi.get_lineage(taxid)
        # remove root taxonomy id
        tax_lineage = [element for element in tax_lineage1 if element not in [1, 2, 131567, 10239]]
        res = []
        inner_count = 0
        for ite in tax_lineage:
            try:
                tmp = tokenizer.taxa_to_ctrl_idx[ite]
                res.append(tmp)
            except:
                inner_count += 1
        if res == [] and inner_count >= 1:
            print("ALERT: error in taxonmy conversions for entry: ", taxid)
        tax_lineage = res
    except:
        print("FATAL taxa conversion for entry: ", taxid)
        # tax_lineage = tokenizer.taxa_to_lineage[taxid] # make lineage in ncbi ids
        # tax_lineage = [tokenizer.taxa_to_ctrl_idx[ite] for ite in tax_lineage] # now translated as ctrl code indices
    '''
    # FOR TESTIG the code:
    # input_seq = input_seq[0:24]
    
    # if no tax keys:
    #tax_lineage = [0]
    #print(input_seq)
    keys = [int(item) for item in entry['kw']] 
    #keys = []
    
    #try:
    res, tokens_prob, offset = generator.after_n_generation(input_seq, keys, 20)
    #except:
    #    print("ALERT: skipped for error the entry: ", entry['metadata']['accession'])
    #    continue
    # print("Input: ", input_seq)
    # print("Res: ", res)
    true_value.append(input_seq)
    true_tokens_index_in_prob = []
    true_tokens_index_in_probs_all.append(true_tokens_index_in_prob)
    tokens_probs_all.append(tokens_prob)
    predicted.append(input_seq[:offset] + res)

    # FOR TESTIG:
    